In [0]:
session = sagemaker.Session() # Store the current SageMaker session
prefix = 'sentiment-prediction-app'

locTest = session.upload_data(os.path.join(dir, 'test.csv'), key_prefix=prefix)
locV = session.upload_data(os.path.join(dir, 'validation.csv'), key_prefix=prefix)
locTrain = session.upload_data(os.path.join(dir, 'train.csv'), key_prefix=prefix)

from sagemaker import get_execution_role
role = get_execution_role()

#retrieve the location of the container (the training and inference code both use the same container)
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(session.boto_region_name, 'xgboost')

rtModel = sagemaker.estimator.Estimator(container, role, train_instance_count=1, train_instance_type='ml.m4.xlarge',      
                                    output_path='s3://{}/{}/output'.format(session.default_bucket(), prefix),
                                    sagemaker_session=session)

# initialize with parameters:
rtModel.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        early_stopping_rounds=10,
                        num_round=500)

s3_input_train = sagemaker.s3_input(s3_data=locTrain, content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data=locV, content_type='csv')

rtModel.fit({'train': s3_input_train, 'validation': s3_input_validation})